In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats
from scipy.stats import gumbel_r

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
jd_offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':jd_offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
jd_collection = []
for cset in clickset_list:
    if cset not in jd_collection:
        jd_collection.append(cset)
print('there are {} different assortments'.format(len(jd_collection)))
print('check offertimes',len(jd_offertimes))

there are 9 different products
there are 134 different assortments
check offertimes 134


In [4]:
def collection_distribution_hev(n,collection,price):
    # generate mean 
    ''' mu_0 = np.random.uniform(2,3)
    mu_1n = np.random.uniform(-3,-2,n-1) '''
    # generate deterministic utilities for products
    
    rho = -0.5 # prices and utilities are negatively correlated
    price_mean = np.mean(price[1:]) 
    price_std = np.std(price[1:])
    
    z = np.zeros(len(price[1:]))
    for i in range(len(z)):
        z[i] = (price[1:][i] - price_mean)/price_std
        
    mu_1n = np.zeros(len(z))
    normal_rvs = np.random.randn(len(mu_1n))
    for i in range(len(mu_1n)):
        mu_1n[i] = rho*z[i] + np.sqrt(1-rho**2)*normal_rvs[i] 
    ## deterministic utility of the outside option is strictly greater than the utilities of the products
    mu_0 = np.max(mu_1n) + 1
    # np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    
    scale_0 = 10
    scale_1n = np.random.uniform(0.04,1,n-1)
    scales = np.hstack((scale_0,scale_1n))
    
    sample_size = 10000
    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])
    # generate covariance matrix with positive correlation
    # neg_cov = generate_negatively_correlated_covariance_matrix(n)
    # if is_positive_semidefinite(neg_cov): 
    #     sample_size = 10000
    #     utility_samples = np.random.multivariate_normal(mu,neg_cov,size=sample_size)
    # else:
    #     print('Negative correlation matrix error')
    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 

        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution

In [5]:
def whole_instance_generation(n,whole_collection,whole_offertimes,price):
    ## generate LCMNL instances based on the assortment collection infomation
    # step 1: randomly generate LCMNL parameters
    # 1.1 the number of classes in LCMNL, the number of classes is between 10 and 15, both endpoints are included
    ''' num_classes = 20
    # 1.2 randomly generate weights of all classes 
    #weight_pre = np.random.exponential(1, num_classes)
    weights = np.array([1/ num_classes for _ in range(num_classes)])
    # 1.3 randomly generate parameters of each mnl
    parameters_v = np.random.uniform(-30, 30, size=(num_classes,n)) '''
    ''' parameters_v = np.zeros((num_classes,n))
    for i in range(num_classes):
        parameters_v[i] = np.random.exponential(1, n) '''
    #np.random.uniform(-30, 30, size=(num_classes,n))
    # step 2: generate LCMNL instance with the above parameters of LCMNL
    true_hev_instance = collection_distribution_hev(n,whole_collection,price)
    # step 3: generate multinomial samples based on lcmnl instance and the emprical assortment offertimes
    purchased_samples = []
    for i in range(len(whole_collection)):
        sample_i = np.random.multinomial(whole_offertimes[i], true_hev_instance[:,i])
        purchased_samples.append(sample_i)
    # step 4: compute the simulated collection probabilities
    whole_choice_collection = np.zeros((n,len(whole_collection)))
    for i in range(len(whole_collection)):
        whole_choice_collection[:,i] = np.array([k/np.sum(purchased_samples[i]) for k in purchased_samples[i]])
        
    return whole_choice_collection,purchased_samples

In [6]:
def filter_collection_offertimes(whole_collection,whole_offertimes,least_offetimes):
    
    collection = []
    offertimes = []
    assortment_index = []
    
    for i in range(len(whole_offertimes)):
        if whole_offertimes[i]>=least_offetimes:
            collection.append(whole_collection[i])
            offertimes.append(whole_offertimes[i])
            assortment_index.append(i)
            
    return collection,offertimes,assortment_index

In [7]:
pred_offer_times_list = [20,30,40,50,60]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
pred_instance_size = [50,50,50,50,50]
price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

In [8]:

all_full_collections = []
all_full_offertimes = []
all_full_assortment_index = []
full_collection_size = []
for i in range(len(pred_offer_times_list)):
    collection, offertimes, assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[i])
    all_full_collections.append(collection)
    all_full_offertimes.append(offertimes)
    all_full_assortment_index.append(assortment_index)
    full_collection_size.append(len(assortment_index))
    print("number of assortments with offertimes {} is {}".format(pred_offer_times_list[i], len(assortment_index)))

number of assortments with offertimes 20 is 29
number of assortments with offertimes 30 is 24
number of assortments with offertimes 40 is 19
number of assortments with offertimes 50 is 15
number of assortments with offertimes 60 is 13


In [9]:
# check if the collection are nested
def check_subsets(lists):
    # Convert all lists to sets
    sets = [set(lst) for lst in lists]
    
    # Check subsets
    for i in range(len(sets) - 1):
        if not sets[i + 1].issubset(sets[i]):
            return False
    return True

# Check and print the result
result = check_subsets(all_full_assortment_index)
print("All subsequent lists are subsets of the previous one:", result)

All subsequent lists are subsets of the previous one: True


In [10]:
def filter_probability_frequency2(whole_choice_collection,purchase_samples,assortment_index):
    
    choice_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    frequency_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    for i in range(len(assortment_index)):
        choice_collection[:,i] = whole_choice_collection[:,assortment_index[i]]
        frequency_collection[:,i] = purchase_samples[assortment_index[i]]
        
    return choice_collection,frequency_collection

In [11]:
# generate instances that satisfy for assortment offertimes is >=20
## generate 50 random full instances 
# the observations with at least 20 times are >= 1e-3
pred_full_instance = []
pred_full_samples = []
#collection_20,offertimes_20,assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[0])

for j in range(pred_instance_size[0]):
    print(f'generating {j} th instance')
    # full instance generation 
    curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes,price)
    # filter choice probability and purchase frequency of each produt in each assortment
    curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])
    
    pred_full_instance.append(curr_whole_instance)
    pred_full_samples.append(curr_purchase_samples)
    #print(pd.DataFrame(curr_whole_instance))
    
    ''' condition = False
    for x in range(len(all_full_collections[0])):
        for y in all_full_collections[0][x]:
            if curr_choice_collection[y][x]<1e-6 or curr_choice_collection[y][x] >0.999:
                condition = True
    while condition:
        curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes,price)
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

        condition = False
        for x in range(len(all_full_collections[0])):
            for y in all_full_collections[0][x]:
                if curr_choice_collection[y][x] < 1e-6 or curr_choice_collection[y][x] >0.999:
                    condition = True
    if condition == True:
        print('instance_generation error')
    else:  
        pred_full_instance.append(curr_whole_instance)
        pred_full_samples.append(curr_purchase_samples)    '''

generating 0 th instance


generating 1 th instance
generating 2 th instance
generating 3 th instance
generating 4 th instance
generating 5 th instance
generating 6 th instance
generating 7 th instance
generating 8 th instance
generating 9 th instance
generating 10 th instance
generating 11 th instance
generating 12 th instance
generating 13 th instance
generating 14 th instance
generating 15 th instance
generating 16 th instance
generating 17 th instance
generating 18 th instance
generating 19 th instance
generating 20 th instance
generating 21 th instance
generating 22 th instance
generating 23 th instance
generating 24 th instance
generating 25 th instance
generating 26 th instance
generating 27 th instance
generating 28 th instance
generating 29 th instance
generating 30 th instance
generating 31 th instance
generating 32 th instance
generating 33 th instance
generating 34 th instance
generating 35 th instance
generating 36 th instance
generating 37 th instance
generating 38 th instance
generating 39 th inst

In [12]:
''' for x in range(len(all_full_collections[0])):
    for y in all_full_collections[0][x]:
        if curr_choice_collection[y][x]<1e-6 or curr_choice_collection[y][x] >0.999:
            print(f'collection {all_full_collections[0][x]} product {y} with prob. {curr_choice_collection[y][x]}') '''

" for x in range(len(all_full_collections[0])):\n    for y in all_full_collections[0][x]:\n        if curr_choice_collection[y][x]<1e-6 or curr_choice_collection[y][x] >0.999:\n            print(f'collection {all_full_collections[0][x]} product {y} with prob. {curr_choice_collection[y][x]}') "

In [13]:
def compute_lb_ub_w_ci(frequency_collection,choice_collection,target_z_score):
    
    stardard_error = np.zeros(frequency_collection.shape)
    for i in range(stardard_error.shape[0]):
        for j in range(stardard_error.shape[1]):
            if frequency_collection[i][j]!=0:
                stardard_error[i][j] = np.sqrt((1-choice_collection[i][j])/frequency_collection[i][j])
                
    lb = np.zeros(frequency_collection.shape)
    ub = np.zeros(frequency_collection.shape)
    for i in range(lb.shape[0]):
        for j in range(lb.shape[1]):
            if frequency_collection[i][j]!=0:
                lb[i][j] = choice_collection[i][j] * (1-target_z_score*stardard_error[i][j]) 
                ub[i][j] = choice_collection[i][j] * (1+target_z_score*stardard_error[i][j])
    
    return lb,ub

In [14]:
# full instance generation 
all_full_instances = []
all_full_lb = []
all_full_ub = []

# pre-determined confidence interval 
confidence_level = 0.995
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)  

for i in range(len(pred_offer_times_list)):
    
    full_instances = []
    full_lbs = []
    full_ubs = []
    print(f'check assortment index for offertimes {pred_offer_times_list[i]}')
    for j in range(pred_instance_size[i]):
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(pred_full_instance[j],pred_full_samples[j],all_full_assortment_index[i])
        # re-check if all choice probability are >= 1e-3
        ''' condition = False
        for x in range(len(all_full_collections[i])):
            for y in all_full_collections[i][x]:
                if curr_choice_collection[y][x] < 1e-11:
                    condition = True
        if condition == True:
            print('instance_generation error') '''
        
        # compute the collection of lower bound and upper bound l_ij and u_ij
        curr_lb,curr_ub = compute_lb_ub_w_ci(curr_frequency,curr_choice_collection,target_z_score)
        
        full_instances.append(curr_choice_collection)
        full_lbs.append(curr_lb)
        full_ubs.append(curr_ub)
    
    all_full_instances.append(full_instances)
    all_full_lb.append(full_lbs)
    all_full_ub.append(full_ubs)
    



check assortment index for offertimes 20
check assortment index for offertimes 30
check assortment index for offertimes 40
check assortment index for offertimes 50
check assortment index for offertimes 60


In [15]:
# train-test split
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(all_full_collections[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(all_full_collections[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(all_full_collections[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = all_full_instances[i][j][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = all_full_lb[i][j][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = all_full_ub[i][j][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(all_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(all_full_collections[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = all_full_instances[i][j][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
        #print('chosen training indexes',chosen_train_idx)
        #print('chosen testing indexes',chosen_test_idx)
        #print('curr train collection\n',curr_train_collection)
        #print('curr test collection\n',curr_test_collection)
        df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst)
        #print('curr train instance\n',df_train)
        #print('curr test instance\n',df_test)
        
        ''' ## output the training and testing instance
        df_train.to_csv('instances/train_instances/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_test.to_csv('instances/test_instances/test_'+str(pred_offer_times_list[i])+'/test_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_offertimes.to_csv('instances/train_offertimes/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
         '''
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [16]:
def mnl_feasibility(data,collection,n):
    
    # MNL cannot reproduce zero probability 
    start_time = time.time()
    for i in range(len(collection)):
        for j in collection[i]:
            if data[j][i] == 0:
                return [0,time.time()-start_time]
    
           
    actual_prod = np.unique(np.nonzero(data)[0]).tolist()
    model = gp.Model('mnl')
    model.setParam('OutputFlag', 0)
    v = model.addVars(n,name = 'v') # positive 
  
    for ass_idx in range(len(collection)):
        for i in collection[ass_idx]:
            model.addConstr(data[i][ass_idx]*sum(v[j] for j in collection[ass_idx])-v[i]==0)
    
    model.addConstrs(v[i]>=0 for i in range(n))
    model.addConstrs(v[i]<=100 for i in range(n))
    #model.setObjective(0,GRB.MAXIMIZE)
    model.setObjective(sum(v[i] for i in actual_prod),GRB.MAXIMIZE)
    model.optimize()
        
    if model.Status == 2: #optimal
        soln = []
        for v in model.getVars():
            soln.append(v.x)
        
        for i in actual_prod:
            if soln[i] == 0:
                return [0,model.Runtime]
        if sum(soln) >0:
            return [1,model.Runtime]

In [17]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

def mnl_mle_pyo(data, collection,offertimes):
    n = data.shape[0]
    model = pyo.ConcreteModel()
    model.v = pyo.Var(range(n), bounds=(-100, 100))
    
    def obj_rule(model):
        obj = 0
        for i in range(len(collection)):
            for j in collection[i]:
                obj += offertimes[i]*data[j][i] * model.v[j]
            obj -= offertimes[i]*pyo.log(sum([pyo.exp(model.v[k]) for k in collection[i]]))
        return obj
    
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)
    
    # scale the preference of the outside option to be 1
    model.outsideopt_constraint = pyo.Constraint(expr=model.v[0] == 0)
    
    opt = pyo.SolverFactory('ipopt')  # use iterior point method to solve convex optimizatio problem 
    results = opt.solve(model)
    
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        v_values = [model.v[i].value for i in range(n)]
        obj_value = model.obj()
        #model.pprint()
        return v_values, obj_value,results.solver.time
    else:
        raise ValueError("Solver failed to find an optimal solution.")

In [18]:
def mnl_distribution(v, collection):
    collection_distribution = np.zeros((len(v), len(collection)))
    for i, curr_collection in enumerate(collection):
        curr_total_v = np.sum([np.exp(v[j]) for j in curr_collection])
        for k, idx in enumerate(curr_collection):
            collection_distribution[idx][i] = np.exp(v[idx]) / curr_total_v
    
    return collection_distribution

In [19]:
## prediction for the current testing collection of assortments 
def mnl_prediction(curr_test_instance,curr_test_collection,price,v_values):
    ''' true assortment ranking '''
    true_choice_prob = [] 
    true_revenue = []
    
    mnl_revenue = []
    mnl_sales = []
    
    mnl_runtime = []
    
    
    # mle estimate
    start_time = time.time()
    estimated_oos_prob = mnl_distribution(v_values,curr_test_collection)
    used_t_1 = time.time() - start_time
    
    for k in range(len(curr_test_collection)):

        # true revenue and true choice probabilities for current testing assortment
        curr_revenue = np.dot(curr_test_instance[:,k],price)
        true_revenue.append(curr_revenue)
        true_choice_prob.append(curr_test_instance[:,k])
        
        st_time_2 = time.time()
        curr_mnl_rev = np.dot(estimated_oos_prob[:,k],price)
        used_t_2 = time.time() - st_time_2
        
        mnl_sales.append(estimated_oos_prob[:,k])
        mnl_revenue.append(curr_mnl_rev)
        print(f'revenue of assortment {curr_test_collection[k]} is {curr_mnl_rev}')
        
        ## record the runtime result
        mnl_runtime.append(used_t_1+used_t_2)
        
    return true_revenue,true_choice_prob,mnl_revenue,mnl_sales,mnl_runtime

In [20]:
def kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    num = len(values1)
    assert len(values2) == num, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(num), np.arange(num))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered/2 

In [21]:
## record representability of all instances

## record the kendall tau distance between the predicted ranking of mnl and the truth
all_mnl_distance_list = []  

## record the true revenue of the predicted best assortment
all_best_true_rev_list = []
all_best_rev_mnl_list = []  

## record the difference in best assortment
all_mnl_best_rev_diff = []

## record all the true and predicted revenues
all_true_revenue_list = []
all_mnl_revenue_list = []

all_true_prob_list = [] 
all_mnl_sales_list = []

all_true_ranking = []
all_mnl_ranking = []

## record the runtime of mnl prediction
all_mnl_pred_runtime = []

for i in range(len(all_full_collections)):
       
    # define as a container for quantity of the same collection size 
    # The following lists are of the same size as the instance size
    
    true_best_rev_collection = [] # a container for groundtruth best reveue 
    mnl_best_rev_collection = [] # a container for the best revenue predicted by robust mnl
    
    true_ranking_collection = [] # a container for groundtruth reveue ranking 
    mnl_ranking_collection = [] # a container for ranking by robust mnl
    
    mnl_distance_collection = [] # a container for kendall tau distance between true ranking and the ranking by robust mnl
   
    
    true_choice_prob_collection = []
    mnl_sales_frac_collection = []

    true_revenue_collection = []
    mnl_revenue_collection = []
    
    mnl_runtime_collection = []
    
    for j in range(pred_instance_size[i]):
        
        ## mle 
        print(f'testing for offertimes {pred_offer_times_list[i]}, {j}th instance')
        
        v_values,ll_value,ll_time = mnl_mle_pyo(all_train_instances[i][j],all_train_collection[i][j],all_train_offertimes[i][j])
        ########### mnl prediction#################
        # mnl_prediction(curr_test_instance,curr_test_collection,price,v_values)
        true_revenue,true_choice_prob,mnl_revenue,mnl_sales,mnl_runtime = mnl_prediction(all_test_instance[i][j],all_test_collection[i][j],price,v_values)
        # the following lists are of the same size as number of testing assortments
        
        for x in range(len(mnl_runtime)):
            mnl_runtime[x] = mnl_runtime[x] + ll_time
        
        true_revenue_collection.append(true_revenue)
        true_choice_prob_collection.append(true_choice_prob)
        
        mnl_revenue_collection.append(mnl_revenue)   
        mnl_sales_frac_collection.append(mnl_sales)
        mnl_runtime_collection.append(mnl_runtime)
         
        # true assortment ranking for test instance [i][j]
        curr_true_rank = np.argsort(-np.array(true_revenue))  
        true_ranking_collection.append(curr_true_rank)
        # true best assortment revenue for test instance [i][j]
        true_best_rev_collection.append(true_revenue[curr_true_rank[0]])
        
        # lb mnl assortment ranking for test instance [i][j] 
        curr_mnl_rank  = np.argsort(-np.array(mnl_revenue)) 
        mnl_ranking_collection.append(curr_mnl_rank)
        mnl_best_rev_collection.append(true_revenue[curr_mnl_rank[0]])
        if true_revenue[curr_mnl_rank[0]]==0 or np.isnan(true_revenue[curr_mnl_rank[0]]):
            print('error in instance {i} {j}')
        
        #  mnl kendallTau Distance for current instance [i][j]
        curr_mnl_dist = kendall_tau_distance(curr_true_rank, curr_mnl_rank)
        mnl_distance_collection.append(curr_mnl_dist)
    
    
    # for each test instance [i][j] kendall tau distance is a number
    all_mnl_distance_list.append(mnl_distance_collection)
    
    # for each test instance [i][j] best revenue is a number
    all_best_true_rev_list.append(true_best_rev_collection) 
    all_best_rev_mnl_list.append(mnl_best_rev_collection)
    
    # for each test instance [i][j] best revenue difference is a number
    with np.errstate(divide='ignore', invalid='ignore'):
        result = (np.array(true_best_rev_collection) - np.array(mnl_best_rev_collection))/ np.array(true_best_rev_collection)
        # Replace NaN values (0/0) with 0 values with a large number or zero
        result = np.where(np.isnan(result), 0.0, result)
        

    all_mnl_best_rev_diff.append(result)
    
    # for each test instance [i][j], true revenue is a list of the same size as the number of testing assortments
    all_true_revenue_list.append(true_revenue_collection)
    
    all_mnl_revenue_list.append(mnl_revenue_collection)
    
    all_true_prob_list.append(true_choice_prob_collection)
    all_mnl_sales_list.append(mnl_sales_frac_collection)
 
    # for each test instance [i][j], true ranking is a list of the same size as the number of testing assortments
    all_true_ranking.append(true_ranking_collection)
    all_mnl_ranking.append(mnl_ranking_collection)

    ## for each test instance [i][j], runtime is a list of the same size as the number of testing assortments
    all_mnl_pred_runtime.append(mnl_runtime_collection)

testing for offertimes 20, 0th instance
revenue of assortment [0, 5] is 0.23359209156186797
revenue of assortment [0, 3, 7] is 0.25362960179481264
revenue of assortment [0, 5, 6] is 0.2695089920760977
revenue of assortment [0, 5, 8] is 0.4532698184760048
revenue of assortment [0, 6, 8] is 0.3350486090726129
testing for offertimes 20, 1th instance
revenue of assortment [0, 3] is 0.13423770635293028
revenue of assortment [0, 4] is 0.23213160361065271
revenue of assortment [0, 1, 4] is 0.4158687399733908
revenue of assortment [0, 3, 4] is 0.28140294659133736
revenue of assortment [0, 5, 6] is 0.2773767482918307
testing for offertimes 20, 2th instance
revenue of assortment [0, 1, 8] is 0.6345096649012396
revenue of assortment [0, 2, 6] is 0.1851887724355952
revenue of assortment [0, 3, 4] is 0.3555841203108178
revenue of assortment [0, 4, 5] is 0.5085781674885324
revenue of assortment [0, 5, 8] is 0.6157248838263081
testing for offertimes 20, 3th instance
revenue of assortment [0, 1] is 0.

In [22]:
## adding details 1
for i in range(len(pred_offer_times_list)):
    df_mnl_details1 = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mnl_dist':all_mnl_distance_list[i],
                                    'true_best_rev':all_best_true_rev_list[i],'mnl_best_rev':all_best_rev_mnl_list[i],
                                    'mnl_best_rev_diff':all_mnl_best_rev_diff[i]})
    df_mnl_details1.to_csv('prediction/mnl/revenue_prediction/details1/'+str(pred_offer_times_list[i])+'.csv')

In [23]:
## adding details 2 about ranking, revenue, choice probability
for i in range(len(pred_offer_times_list)):
    for j in range(pred_instance_size[i]):
        
        true_rank = all_true_ranking[i][j]
        mnl_rank = all_mnl_ranking[i][j]
        df_rank = pd.DataFrame({'true_rank':true_rank,'mnl_rank':mnl_rank})
        df_rank.to_csv('prediction/mnl/revenue_prediction/details2/ranking/offertimes'+str(pred_offer_times_list[i])+'/rank_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        true_rev = all_true_revenue_list[i][j]
        mnl_rev = all_mnl_revenue_list[i][j]
        df_rev = pd.DataFrame({'true_rev':true_rev,'mnl_rev':mnl_rev})
        df_rev.to_csv('prediction/mnl/revenue_prediction/details2/revenue/offertimes'+str(pred_offer_times_list[i])+'/rev_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
    
        mnl_runtime = all_mnl_pred_runtime[i][j]
        df_runtime = pd.DataFrame({'mnl_runtime':mnl_runtime})
        df_runtime.to_csv('prediction/mnl/revenue_prediction/details2/runtime/offertimes'+str(pred_offer_times_list[i])+'/runtime_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        ## using sales fraction prediction
        ## record the choice probability prediction interval
        ''' true_prob = all_true_prob_list[i][j]
        df_true_prob = pd.DataFrame(true_prob)
        
        mnl_sales = all_mnl_sales_list[i][j]
        df_mnl_sales = pd.DataFrame(mnl_sales).T
 
        df_true_prob.to_csv('prediction/mnl/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/true/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_mnl_sales.to_csv('prediction/mnl/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/mnl/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''

In [24]:
## avg prediction mnl runtime 
all_mnl_pred_avg_runtime = []

for i in range(len(pred_offer_times_list)):
    avg_mnl_pred_runtime_collection = []
    
    for j in range(pred_instance_size[i]): 
        avg_mnl_pred_runtime_collection.append(np.mean(np.array(all_mnl_pred_runtime[i][j]))) 
        
    all_mnl_pred_avg_runtime.append(np.mean(np.array(avg_mnl_pred_runtime_collection)))

avg_mnl_dist = []
avg_mnl_best_rev_diff = []

for i in range(len(pred_offer_times_list)):
    
    avg_mnl_dist.append(np.mean(all_mnl_distance_list[i]))
    avg_mnl_best_rev_diff.append(np.mean(all_mnl_best_rev_diff[i]))
    

df_mnl_jd_summary = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'avg_mnl_dist':avg_mnl_dist,'avg_mnl_best_rev_diff':avg_mnl_best_rev_diff,
                                   'avg_mnl_pred_runtime':all_mnl_pred_avg_runtime})
df_mnl_jd_summary.to_csv('prediction/mnl/jd_mnl_summary.csv')

In [25]:
df_mnl_jd_summary

,offertimes,avg_mnl_dist,avg_mnl_best_rev_diff,avg_mnl_pred_runtime
0,20,3.44,0.150055,0.229203
1,30,2.16,0.210690,0.229024
2,40,0.82,0.127340,0.227871
3,50,0.90,0.146483,0.242559
4,60,0.26,0.087411,0.259189


In [26]:
## record representability of all instances
all_mnl_rep = []
all_mnl_rep_time = []

## record the limit loss of all instances
all_mnl_limit_loss = []
all_mnl_limit_prob = []
all_mnl_limit_runtime = []

for i in range(len(pred_offer_times_list)):
    
    ## define container for mnl representability check
    mnl_rep_collection =[]
    mnl_rep_runtime_collection =[]
    
    ## define container for mnl limit 
    mnl_limit_loss_collection =[]
    mnl_limit_runtime_collection =[]
    mnl_limit_probability_collection =[]
    
    for j in range(pred_instance_size[i]):
        
        ####### mnl representability check ##########
        
        # mnl_feasibility(data,collection,n)
        curr_rep_result = mnl_feasibility(all_full_instances[i][j],all_full_collections[i],n)
        # return [1/0,model.Runtime] # 1: representable 0: non-representable
        mnl_rep_collection.append(curr_rep_result[0])
        mnl_rep_runtime_collection.append(curr_rep_result[1])
        
        if curr_rep_result[0] > 0:
           
            # representable - no loss
            mnl_limit_probability_collection.append(all_full_instances[i][j])
            mnl_limit_loss_collection.append(0)
            mnl_limit_runtime_collection.append(0)
            
        else:
            # not representable # limit 
            start_time = time.time()
            v_values1,ll_value1,ll_time1 = mnl_mle_pyo(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])
            # return v_values, obj_value,results.solver.time
            
            curr_limit_prob = mnl_distribution(v_values1, all_full_collections[i])
            curr_loss = 0
            for x in range(len(all_full_collections[i])):
                for y in all_full_collections[i][x]:
                    curr_loss = curr_loss + all_full_offertimes[i][x]*all_full_instances[i][j][y][x]*np.abs(all_full_instances[i][j][y][x] -curr_limit_prob[y][x])
            #curr_loss = np.sum(all_full_instances[i][j] * np.abs(all_full_instances[i][j] - curr_limit_prob))
            used_time = time.time() - start_time  
            
            mnl_limit_probability_collection.append(curr_limit_prob)
            mnl_limit_loss_collection.append(curr_loss)
            mnl_limit_runtime_collection.append(used_time)
    ## for each train instance [i][j] representability result is a number
    all_mnl_rep.append(mnl_rep_collection)
    all_mnl_rep_time.append(mnl_rep_runtime_collection)
    
    ## for each train instance [i][j] limit loss result is a number
    all_mnl_limit_loss.append(mnl_limit_loss_collection)
    all_mnl_limit_runtime.append(mnl_limit_runtime_collection)
    
    ## for each train instance [i][j] limit probability is a matrix
    all_mnl_limit_prob.append(mnl_limit_probability_collection)

Academic license - for non-commercial use only - expires 2024-12-10
Using license file /Users/autumn/gurobi.lic


In [27]:
## adding rep results
ins_idx = list(range(pred_instance_size[0]))
for i in range(len(pred_offer_times_list)):
    df_mnl_rep = pd.DataFrame({'ins_idx':ins_idx,'mnl_rep':all_mnl_rep[i], 'mnl_rep_time':all_mnl_rep_time[i]})
    df_mnl_rep.to_csv('limit/mnl/representability/'+str(pred_offer_times_list[i])+'.csv')

In [28]:
## adding limit results
ins_idx = list(range(pred_instance_size[0]))
for i in range(len(pred_offer_times_list)):
    df_mnl_loss = pd.DataFrame({'ins_idx':ins_idx,'mnl_loss':all_mnl_limit_loss[i],'avg_loss':all_mnl_limit_loss[i]/sum(all_full_offertimes[i]),  'mnl_limit_time':all_mnl_limit_runtime[i]})
    df_mnl_loss.to_csv('limit/mnl/limit/'+str(pred_offer_times_list[i])+'.csv')
    
    for j in range(pred_instance_size[i]):
        df_limit_prob = pd.DataFrame(all_mnl_limit_prob[i][j])
        df_limit_prob.to_csv('limit/mnl/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')

In [29]:
all_avg_loss = []
for k in range(len(pred_offer_times_list)):
    all_avg_loss.append(all_mnl_limit_loss[k]/sum(all_full_offertimes[k]))

In [30]:
avg_mnl_rep = []
avg_mnl_rep_runtime = []

avg_total_limit_loss = []
avg_toal_limit_loss_se = []
avg_mnl_limit_runtime = []

#avg_loss = []
avg_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_mnl_rep.append(np.mean(all_mnl_rep[i]))
    avg_mnl_rep_runtime.append(np.mean(all_mnl_rep_time[i]))
    
    avg_total_limit_loss.append(np.mean(all_mnl_limit_loss[i]))
    avg_mnl_limit_runtime.append(np.sum(all_mnl_limit_runtime[i])/(pred_instance_size[i] - np.sum(all_mnl_rep[i])))
    avg_toal_limit_loss_se.append(np.std(all_mnl_limit_loss[i])/np.sqrt(len(all_mnl_limit_loss[i])))
    
    #avg_loss = np.mean(all_avg_loss[i])
    avg_loss_se.append(np.std(all_avg_loss[i])/np.sqrt(len(all_avg_loss[i])))

In [31]:
avg_loss = []
for i in range(len(avg_total_limit_loss)):
    avg_loss.append(avg_total_limit_loss[i]/sum(all_full_offertimes[i]))

In [32]:
df_mnl_limit = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'mnl_rep':avg_mnl_rep,'mnl_rep_runtime':avg_mnl_rep_runtime,
                                   'total_mnl_loss':avg_total_limit_loss,'total_mnl_loss_se':avg_toal_limit_loss_se,
                                   'avg_mnl_loss':avg_loss,'avg_mnl_loss_se':avg_loss_se,
                                   'avg_mnl_limit_runtime':avg_mnl_limit_runtime
                                   })
df_mnl_limit.to_csv('limit/mnl/mnl_limit_summary.csv')
df_mnl_limit

,offertimes,mnl_rep,mnl_rep_runtime,total_mnl_loss,total_mnl_loss_se,avg_mnl_loss,avg_mnl_loss_se,avg_mnl_limit_runtime
0,20,0.0,0.000027,280.884538,4.529448,0.036693,0.000592,0.264386
1,30,0.0,0.000052,255.514148,4.176252,0.033937,0.000555,0.240225
2,40,0.0,0.000064,224.508997,3.709734,0.030500,0.000504,0.248980
3,50,0.0,0.000102,190.171136,3.621666,0.026449,0.000504,0.288568
4,60,0.0,0.000176,166.524835,3.381099,0.023514,0.000477,0.316495


In [33]:
for i in range(len(avg_total_limit_loss)):
    print(avg_total_limit_loss[i]/sum(all_full_offertimes[i]))

0.03669295072540327
0.03393732866722501
0.030499795821076602
0.02644939303328926
0.0235138145936594


### compute the kl loss for the limit

In [41]:
all_mnl_kl_loss = []
all_mnl_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    mnl_kl_loss_collection = []
    mnl_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        limit_prob = all_mnl_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y])
        mnl_kl_loss_collection.append(-curr_kl_loss)
        mnl_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    df_mnl_loss = pd.DataFrame({'ins_idx':ins_idx,'mnl_loss':mnl_kl_loss_collection,'avg_loss':mnl_avg_kl_loss_collection})
    df_mnl_loss.to_csv('limit/mnl/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_mnl_kl_loss.append(mnl_kl_loss_collection) 
    all_mnl_avg_kl_loss.append(mnl_avg_kl_loss_collection)
        

In [42]:
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mnl_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mnl_kl_loss[i])/np.sqrt(len(all_mnl_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mnl_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mnl_avg_kl_loss[i])/np.sqrt(len(all_mnl_avg_kl_loss[i])))
    
df_mnl_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mnl_kl_loss':avg_total_kl_loss,'total_mnl_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mnl_kl_loss':avg_avg_kl_loss,'avg_mnl_kl_loss_se':avg_avg_kl_loss_se
                                   })
df_mnl_kl.to_csv('limit/mnl/mnl_kl_summary.csv')
df_mnl_kl 

,offertimes,total_mnl_kl_loss,total_mnl_kl_loss_se,avg_mnl_kl_loss,avg_mnl_kl_loss_se
0,20,4902.571443,25.673560,0.640440,0.003354
1,30,4796.238862,25.068785,0.637035,0.003330
2,40,4657.732967,24.041029,0.632758,0.003266
3,50,4515.272053,23.336593,0.627993,0.003246
4,60,4424.727123,22.922449,0.624785,0.003237
